In [1]:
import sys
import pandas as pd
from pathlib import Path
import os
import random
from time import gmtime, strftime

chexpert_dir = '/scratch/wboag/2019/cxr/baselines_experiments/chexpert-labeler'
if chexpert_dir not in sys.path:
    sys.path.append(chexpert_dir)

from loader import Loader
from stages import Extractor, Classifier, Aggregator

In [2]:
def mkpath(path_str, posix=True):
    path = os.path.join(chexpert_dir, path_str)
    if posix:
        return Path(path)
    else:
        return path

In [3]:
# Load reports
df_knn = pd.read_csv('/scratch/wboag/2019/cxr/cxr-baselines_experiments/camera_ready/output/knn.tsv', sep='\t')

print(df_knn.shape)
df_knn.head()

In [4]:
# Write reports to temporary file
tempname = '/tmp/chexpert-reports-%s.csv' % random.randint(0,10**6)
print(tempname)
with open(tempname, 'w') as f:
    for text in df_knn.generated.values:
        print(text.replace(',', '\\,'), file=f)

/tmp/chexpert-reports-220273.csv


In [ ]:
extractor = Extractor(mkpath('phrases/mention'), mkpath('phrases/unmention'), False)
classifier = Classifier(mkpath('patterns/pre_negation_uncertainty.txt', posix=False),
                        mkpath('patterns/negation.txt'                , posix=False),
                        mkpath('patterns/post_negation_uncertainty.txt', posix=False),
                        verbose=True)

CATEGORIES = ["No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly",
              "Lung Lesion", "Lung Opacity", "Edema", "Consolidation",
              "Pneumonia", "Atelectasis", "Pneumothorax", "Pleural Effusion",
              "Pleural Other", "Fracture", "Support Devices"]
aggregator = Aggregator(CATEGORIES, False)

/scratch/cxr-baselines/cxr-baselines/venv_cxr/lib/python3.5/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [ ]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

# Nearest Neighbor
loader = Loader(tempname, False)

# Load reports in place.
loader.load()
# Extract observation mentions in place.
extractor.extract(loader.collection)
# Classify mentions in place.
classifier.classify(loader.collection)
# Aggregate mentions to obtain one set of labels for each report.
labels = aggregator.aggregate(loader.collection)

print(labels.shape)
print(labels[:5])

print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2019-11-14 00:42:26



  0%|          | 0/99145 [00:00<?, ?it/s]

Classifying mentions...


/scratch/cxr-baselines/cxr-baselines/venv_cxr/lib/python3.5/site-packages/StanfordDependencies/JPypeBackend.py:160: UserWarning: This jar doesn't support universal dependencies, falling back to Stanford Dependencies. To suppress this message, call with universal=False
  warnings.warn("This jar doesn't support universal "
  1%|          | 620/99145 [06:00<15:53:53,  1.72it/s]ERROR:root:Cannot process sentence 115 in 620
Traceback (most recent call last):
  File "/scratch/cxr-baselines/cxr-baselines/venv_cxr/lib/python3.5/site-packages/negbio/pipeline/ptb2ud.py", line 119, in convert_doc
    has_lemmas=self.add_lemmas)
  File "/scratch/cxr-baselines/cxr-baselines/venv_cxr/lib/python3.5/site-packages/negbio/pipeline/ptb2ud.py", line 207, in convert_dg
    annotation_id_map[node.head]), 'governor'))
KeyError: 8
  1%|          | 1137/99145 [11:16<16:12:27,  1.68it/s]ERROR:root:Cannot process sentence 197 in 1137
Traceback (most recent call last):
  File "/scratch/cxr-baselines/cxr-baselines

In [ ]:
os.remove(tempname)